In [1]:
import pandas as pd
import numpy as np
import dtale

# Set the notebook to display all columns of a dataframe
pd.set_option('display.max_columns', None)

# import service now data 
inc_df = pd.read_csv(r'do_not_commit\Datasets\SerivceNow_Incident.csv')


C:\Users\JRankin\AppData\Local\Temp\ipykernel_2196\3611318684.py:9: DtypeWarning:

Columns (0,2,3,4,8,23,37,42,47,48,49,57,67,74,80,86,91,92,94,97,101,103) have mixed types. Specify dtype option on import or set low_memory=False.



## Ivestigate machines with more than one incident logged on the same day
We need to be able to join the incidents data with the events data. To do this properly, we will need there to be a many-to-one relationship between events and incidents. Let's evaluate the extent of machines that have more than one incident per day.

In [2]:
def identify_duplicate_machine_groups(inc_df, return_dups_only=True):

    # Reformat date time to date 
    inc_df['opened_at'] = pd.to_datetime(inc_df['opened_at'])
    inc_df['opened_at_formatted'] = inc_df['opened_at'].dt.strftime('%Y-%m-%d')

    # Drop incidents that cannot be tied to a machine
    inc_df = inc_df[inc_df['configuration_item'].notnull()]

    # Investigate duplicate incidents 
    grouped_counts = inc_df.groupby(['opened_at_formatted', 'configuration_item']).size()
    num_inc_machine_dups = grouped_counts[grouped_counts > 1].sum()
    print("Number of incidents that are apart of a duplicate machine-day combination: ", num_inc_machine_dups)

    # identify duplicates 
    grouped_counts = grouped_counts.reset_index().rename(columns={0:'count'})
    grouped_counts['dup'] = grouped_counts['count'].apply(lambda x: '1' if x > 1 else 0)
    grouped_counts['group_index'] = list(range(len(grouped_counts)))
    inc_df2 = pd.merge(inc_df, grouped_counts, how='left', left_on=['opened_at_formatted', 'configuration_item'],
                        right_on=['opened_at_formatted', 'configuration_item'])
    
    if return_dups_only:
        
        # Select only relevant columns for analysis 
        select_columns = ['configuration_item', 'opened_at_formatted', 'number',
                            'category', 'subcategory', 'short_description_NER',
                            'u_cause_code', 'calling_user_id', 'opened_at', 
                            'closed_at', 'severity', 'urgency', 'count', 'dup', 'group_index']
        inc_df2 = inc_df2[select_columns]

        # Output a dataframe showing only duplicates on configuration_item, opened_at groupings
        inc_df2 = inc_df2.query("""`dup` == '1'""")
        inc_df2 = inc_df2.sort_values(['count', 'configuration_item', 'opened_at_formatted'], ascending=[False, True, True])
    
    return inc_df2

# Get the data and view in dtale;
inc_df2 = identify_duplicate_machine_groups(inc_df)
#dtale.show(inc_df2).open_browser()

# or, optionally export to excel and view in a spreadsheet software
# inc_df2.to_excel('do_not_commit/Datasets/duplicate_inc_explore.xlsx', index=False)

Number of incidents that are apart of a duplicate machine-day combination:  416


### Observations for multiple incidents filed on the same day
* In many instances, the multiple incidents appear to be addressing the same problem and introducing redundancy in the dataset. For example, machine name `CHI-L-U31514` on `2023-06-15` has 3 incidents all referring to a missing macabacus add-in for Excel.
* It appears that a user will call the service desk about one issue but afterwards request help with another, potentially unrelated issue. For example, machine name `HIB-L-U29727` on `2023-04-18` had issues with excel and one note not syncing, then requested help with Zoom install. 
* Some of these INC appear to be related to new computers that are experiencing multiple issues. For example, machine name `ENDPOINTIH3UHZ2` on `2023-03-13` was a new laptop but had audio, display, and OneNote syncing issues (hence, multiple INC). 

### Options for path forward on duplicates
1. Given our time constraints, it may be best to simply drop the duplicates on `configuration_item` and `opened_at_formatted`, so we only have one incident per machine per day. The disadvantage with this approach is that we lose detailed information about multiple issues on a machine. 
2. We could combine the textual data for the multiple incidents occurring on the same machine on the same day, such that if there were three incidents, they would all be mapped to one incident. The disadvantage of this approach is that it may not be as easy to sort or search for incidents on `category` or `short_description`. 

I'm going to proceed with option 2 and try to combine the incidents into one record for a given group. 

### Process for mapping multiple INC to one INC:
1. For each group, do the following:
    * Combine `short_description_NER` text into one string, but separate multiple INC `short_description` with a semicolon `;`
    * For groups containing more than 2 INC, grab the most common `category`, `subcategory`, and `u_cause_code`, otherwise grab the first category, subcategory, and u_cause_code.
    * Assign all other values with the attributes associated with the minimum `opened_at` time. 
    * Assign the `closed_at` time with the maximum date for the group. 
2. Drop the duplicate group-related records from the main INC dataset.
3. Combine the output processed INC records with the main INC dataset. 

In [3]:
f = pd.DataFrame({"group":['a','a','a'], "times": ['2023-01-02 16:01:00', '2023-01-02 16:02:00','2023-01-02 16:03:00']})

out_dict = f.groupby('times').size().to_dict()
sorted_dict = {k: v for k, v in sorted(out_dict.items(), key=lambda item: item[1], reverse=True)}
out_val = list(sorted_dict.keys())[0]


In [4]:
def merge_inc_records(in_df):

    # If there are no duplicates, just return the row
    if len(in_df) <= 1:
        out_result = {col: in_df[col].iloc[0] for col in in_df.columns}

    # Process if there are duplicates
    else:
        
        # define output dictionary
        return_dict= {}

        # Get unique short_descriptions and merge them
        short_des = '; '.join(list(in_df['short_description_NER'].unique()))
        return_dict['short_description_NER'] = short_des

        # get the most frequent occurring value for categories and cause code
        for col in ['category', 'subcategory', 'u_cause_code']:

            out_dict = in_df.groupby(col).size().to_dict()
            sorted_dict = {k: v for k, v in sorted(out_dict.items(), key=lambda item: item[1], reverse=True)}
            out_val = list(sorted_dict.keys())[0]
            return_dict[col] = out_val
    
        # Get max close time
        return_dict['closed_at'] = in_df['closed_at'].max()

        # Get record associated with the min open time
        out_df = in_df.loc[in_df['opened_at'].idxmin()]

        # assign output result
        out_result = {}
        for val in out_df.index:

            if val not in return_dict.keys():
                out_result[val] = out_df[val]
            else:
                out_result[val] = return_dict[val]
        
    return pd.Series(out_result, index=list(in_df.columns))


def process_dup_inc(in_df):

    # Get the duplicate groups 
    dup_inc = identify_duplicate_machine_groups(inc_df, return_dups_only=False)

    # change closed_at and opened_at to type datetime
    for val in ['opened_at', 'closed_at']:
        dup_inc[val] = pd.to_datetime(dup_inc[val])

    # Group records and process duplicates
    out_df = dup_inc.groupby('group_index').apply(merge_inc_records)

    return out_df
    
processed_df = process_dup_inc(inc_df)

Number of incidents that are apart of a duplicate machine-day combination:  416


In [5]:
# Perform a spot check on records that were merged 
processed_df.rename(columns={'group_index':'group_index_orig'}, inplace=True)
#dtale.show(processed_df).open_browser()

In [6]:
print('original dataframe length for records that had a machine name: ', len(inc_df[inc_df['configuration_item'].notnull()]))
print('newly processed dataframe length: ', len(processed_df))

original dataframe length for records that had a machine name:  6320
newly processed dataframe length:  6104


In [7]:
# Document dependencies in a jupyter notebook

# Dependencies for this notebook
%load_ext watermark
%watermark
%watermark --iversions

Last updated: 2023-07-29T18:00:29.592361-05:00

Python implementation: CPython
Python version       : 3.11.3
IPython version      : 8.13.1

Compiler    : MSC v.1934 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 154 Stepping 3, GenuineIntel
CPU cores   : 20
Architecture: 64bit

dtale : 3.3.0
pandas: 2.0.3
numpy : 1.25.1

